In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/DR/images"

os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras 

from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
from cv2 import cv2
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt

from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.utils import plot_model
from keras.models import Model
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from numpy import array 
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile

In [7]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
imageList = os.listdir(path)
imageList_clean=[]
for i in range(len(imageList)):
  if imageList[i][-4:] == 'jpeg':
    imageList_clean.append(imageList[i])
  else:
    print(imageList[i])

drive


In [0]:
image=[]
for i in range(len(imageList_clean)):
    image.append(Image.open(path+'/'+imageList_clean[i]))

In [0]:
I_array=[]
for im in image:
  I_array.append(np.array(im))

In [0]:
for i in range(len(I_array)):
  I_array[i] = cv2.resize(I_array[i], (512,512), interpolation=cv2.INTER_CUBIC)


In [0]:
import json
with open('/content/drive/My Drive/DR/labels.json', 'r') as f:
    labels = json.load(f)

In [12]:
import pandas as pd
labeldict=dict(labels)
dataset=pd.DataFrame()
dataset['image_irr']=I_array
dataset['index']=imageList_clean
dataset['label']=dataset['index'].map(labeldict)
dataset.head()

,image_irr,index,label
0,"[[[2, 0, 3], [2, 0, 3], [2, 0, 3], [2, 0, 3], ...",23731_left.jpeg,1
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23538_left.jpeg,2
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23722_right.jpeg,0
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23611_right.jpeg,2
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23717_left.jpeg,2


In [13]:
df_series = pd.Series(dataset['label'])
df_OH = pd.get_dummies(df_series)
df_OH.tail()

,0,1,2,3,4
1495,1,0,0,0,0
1496,1,0,0,0,0
1497,0,0,1,0,0
1498,0,0,1,0,0
1499,1,0,0,0,0


In [14]:
df_all= pd.concat([dataset, df_OH], axis= 1)
df_all.head()

,image_irr,index,label,0,1,2,3,4
0,"[[[2, 0, 3], [2, 0, 3], [2, 0, 3], [2, 0, 3], ...",23731_left.jpeg,1,0,1,0,0,0
1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23538_left.jpeg,2,0,0,1,0,0
2,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23722_right.jpeg,0,1,0,0,0,0
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23611_right.jpeg,2,0,0,1,0,0
4,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",23717_left.jpeg,2,0,0,1,0,0


In [15]:
df_all_fn= df_all.drop('label', axis=1)
df_all_fn=df_all_fn.drop('image_irr',axis=1)
df_all_fn.head()

,index,0,1,2,3,4
0,23731_left.jpeg,0,1,0,0,0
1,23538_left.jpeg,0,0,1,0,0
2,23722_right.jpeg,1,0,0,0,0
3,23611_right.jpeg,0,0,1,0,0
4,23717_left.jpeg,0,0,1,0,0


In [0]:
train, test= train_test_split(df_all, test_size=0.2, random_state=42)

In [17]:
labels_count = dict()
for img_class in range(5):    
    labels_count[img_class] = sum(train[img_class] )

total_count = sum(labels_count.values())
class_weights = {cls: total_count / count for cls, count in 
                 enumerate(labels_count.values())}
class_weights            

{0: 1.3793103448275863,
 1: 11.881188118811881,
 2: 6.486486486486487,
 3: 48.0,
 4: 63.1578947368421}

In [18]:
#data augment
from keras_preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(width_shift_range= 0.2, height_shift_range= 0.2,
    rotation_range= 90, rescale = 1/255,
    horizontal_flip= True, vertical_flip=True)
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="/content/drive/My Drive/DR/images",
x_col="index",
y_col=[0,1,2,3,4],
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
target_size=(512,512))

Found 1200 validated image filenames.


In [19]:
datagen_test=ImageDataGenerator(
    horizontal_flip=True)
test_generator=datagen_test.flow_from_dataframe(
dataframe=test,
directory="/content/drive/My Drive/DR/images",
x_col="index",
y_col=[0,1,2,3,4],
batch_size=32,
seed=42,
shuffle=True,
class_mode="other",
target_size=(512,512))

Found 300 validated image filenames.


In [20]:
visible = Input(shape=(512,512,3))
conv1 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(visible)
conv2 = Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1, 1))(conv1)
bat1 = BatchNormalization()(conv2)
pool1 = MaxPooling2D(pool_size=(2, 2))(bat1)
zero1 = ZeroPadding2D(padding=(1, 1))(pool1)

conv3 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(zero1)
conv4 = Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(conv3)
bat2 = BatchNormalization()(conv4)
pool2 = MaxPooling2D(pool_size=(2, 2))(bat2)

conv5 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(pool2)
conv6 = Conv2D(64, kernel_size=(3,3), activation='relu',strides=(1, 1), padding='valid')(conv5)
bat3 = BatchNormalization()(conv6)
pool3 = MaxPooling2D(pool_size=(2, 2))(bat3)
zero2 = ZeroPadding2D(padding=(1, 1))(pool3)

conv7 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.01))(zero2)
conv8 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv7)
bat4 = BatchNormalization()(conv8)

conv9 = Conv2D(64, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.02))(bat4)
conv10 = Conv2D(64, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.02))(conv9)
bat5 = BatchNormalization()(conv10)

conv11 = Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1, 1))(bat5)
conv12 = Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1, 1))(conv11)
bat6 = BatchNormalization()(conv12)
pool4 = MaxPooling2D(pool_size=(2, 2))(bat6)

conv13 = Conv2D(64, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.02))(pool4)
conv14 = Conv2D(64, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.02))(conv13)
bat7 = BatchNormalization()(conv14)

conv15 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.05))(bat7)
conv16 = Conv2D(128, kernel_size=(2,2), activation='relu', strides=(1, 1), padding='valid')(conv15)
bat8 = BatchNormalization()(conv16)

flat = Flatten()(bat8)
hidden1 = Dense(32, activation='relu')(flat)
drop1 = Dropout(0.3)(hidden1)

hidden2 = Dense(32, activation='relu')(drop1)
drop2 = Dropout(0.2)(hidden2)

output = Dense(5, activation='sigmoid')(drop2)
model = Model(inputs=visible, outputs=output)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 510, 510, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 508, 508, 16)      2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 508, 508, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 16)      0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 254, 254, 32)      4640

In [0]:
opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [24]:
model.compile(optimizer= opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
Callbacks=[EarlyStopping(patience=3, restore_best_weights=True), 
           ReduceLROnPlateau(patience=2), 
           ModelCheckpoint(filepath='ImageDataGen_Size512_oneHOT_ClassWeights_Callbacks_SGD_L2.h5', save_best_only=True)]

In [28]:
with tf.device('/device:CPU:0'): 
  m1= model.fit_generator(
            train_generator,
            epochs=5,
            validation_data=test_generator,
            class_weight=class_weights,
            callbacks=Callbacks,
            verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
38/38 [==============================] - 79s 2s/step - loss: 93.0423 - acc: 0.6678 - val_loss: 123.0948 - val_acc: 0.7500
Epoch 2/5
38/38 [==============================] - 75s 2s/step - loss: 111.8595 - acc: 0.7253 - val_loss: 83.5481 - val_acc: 0.7500
Epoch 3/5
38/38 [==============================] - 76s 2s/step - loss: 75.9383 - acc: 0.7245 - val_loss: 55.1318 - val_acc: 0.7500
Epoch 4/5
38/38 [==============================] - 75s 2s/step - loss: 53.7973 - acc: 0.7261 - val_loss: 38.2010 - val_acc: 0.7500
Epoch 5/5
38/38 [==============================] - 76s 2s/step - loss: 40.7356 - acc: 0.7245 - val_loss: 27.5141 - val_acc: 0.7500


In [29]:
pre_model = load_model('ImageDataGen_Size512_oneHOT_ClassWeights_Callbacks_SGD_L2.h5')
pred = pre_model.predict_generator(test_generator, use_multiprocessing= True, verbose=1)


10/10 [==============================] - 5s 538ms/step


In [30]:
predicted_class_indices = pred
print(np.argmax(test_generator.labels, axis=1))
print(np.argmax(pred, axis=1))

print(sum(np.argmax(test_generator.labels, axis=1) == np.argmax(pred, axis=1)) / len(pred))

[2 0 0 2 1 0 2 0 2 0 2 0 3 0 0 0 0 1 0 0 2 0 0 0 0 0 0 0 0 0 2 0 2 2 0 2 1
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 3 0 0 1 1 0 0 0 0 0 2 0 0 0 1 0 0 0 0 0 0 0 0
 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 2 1 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 3 2 0 0 0 0 0 2 1 0 0 2 2 0 2 2 1 0 0 2 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0
 0 0 0 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 3 1 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 1 0 0 0 0 0 0 0 0 1 0 0 2 0 0 2 1 0 3 2 0 0 0 0 0 0 0 0 2 0 0 0 2
 0 0 0 0 0 0 0 0 0 0 0 3 2 0 0 2 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 0 0
 3 0 0 2 2 0 0 0 0 3 0 0 1 3 0 0 2 0 0 0 0 0 3 2 0 0 0 2 1 0 0 2 2 0 4 0 1
 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 

In [0]:
predicted_class_indices

array([[0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377],
       [0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377],
       [0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377],
       ...,
       [0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377],
       [0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377],
       [0.20613018, 0.21034655, 0.21100241, 0.21388641, 0.20397377]],
      dtype=float32)